In [ ]:
import taskingai
# Load TaskingAI API Key from environment variable

# TaskingAI Model Inference: Chat Completion

In [ ]:
from taskingai.inference import *
import json
# choose an available chat_completion model from your project
model_id = "YOUR_MODEL_ID"

## Single-round chat completion

In [ ]:
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are an assistant specialized in productivity and time management strategies."),
        UserMessage("I'm struggling with managing my time effectively. Do you have any tips?"),
    ]
)
chat_completion_result

## Multi-round chat completion

In [ ]:
# Multiple Round Interaction with Specific System Prompt
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are an assistant with extensive knowledge in literature and book recommendations."),
        UserMessage("Hello, I'm looking for book recommendations."),
        AssistantMessage("Certainly! What genre are you interested in?"),
        UserMessage("I love science fiction."),
        AssistantMessage("Great choice! I recommend 'Dune' by Frank Herbert for its rich world-building and 'Neuromancer' by William Gibson for its cyberpunk influence."),
        UserMessage("Thanks! Can you also suggest something in non-fiction?"),
    ]
)
chat_completion_result

In [ ]:
# Add max tokens configs
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        SystemMessage("You are an assistant with extensive knowledge in literature and book recommendations."),
        UserMessage("Hello, I'm looking for book recommendations."),
        AssistantMessage("Certainly! What genre are you interested in?"),
        UserMessage("I love science fiction."),
        AssistantMessage("Great choice! I recommend 'Dune' by Frank Herbert for its rich world-building and 'Neuromancer' by William Gibson for its cyberpunk influence."),
        UserMessage("Thanks! Can you also suggest something in non-fiction?"),
    ],
    configs={
        "max_tokens": 5
    }
)
chat_completion_result

## Function call

In [ ]:
# function definition
function = Function(
    name="plus_a_and_b",
    description="Sum up a and b and return the result",
    parameters={
        "type": "object",
        "properties": {
            "a": {
                "type": "integer",
                "description": "The first number"
            },
            "b": {
                "type": "integer",
                "description": "The second number"
            }
        },
        "required": ["a", "b"]
    },
)

In [ ]:
# chat completion with the function call
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        UserMessage("What is the result of 112 plus 22?"),
    ],
    functions=[function]
)
function_call_assistant_message = chat_completion_result.message
print(f"function_call_assistant_message = {function_call_assistant_message}")

In [ ]:
# get the function call result
def plus_a_and_b(a, b):
    return a + b

arguments = function_call_assistant_message.function_calls[0].arguments
function_id = function_call_assistant_message.function_calls[0].id
function_call_result = plus_a_and_b(**arguments)
print(f"function_call_result = {function_call_result}")

In [ ]:
# chat completion with the function result
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        UserMessage("What is the result of 112 plus 22?"),
        function_call_assistant_message,
        FunctionMessage(id=function_id, content=str(function_call_result))
    ],
    functions=[function]
)
chat_completion_result

## Stream mode

In [ ]:
# generate in stream mode
chat_completion_result = taskingai.inference.chat_completion(
    model_id=model_id,
    messages=[
        UserMessage("Please count from 1 to 100, and add a new line after every 10 numbers."),
    ],
    stream=True
)
for item in chat_completion_result:
    
    if isinstance(item, ChatCompletionChunk):
        print(item.delta, end="", flush=True)
        
    elif isinstance(item, ChatCompletion):
        print(f"\n\nFinished with reason: {item.finish_reason}")